0. Import

In [ ]:
import pandas as pd
from itertools import islice
from collections import defaultdict
import networkx as nx
from itertools import combinations
import random

1. CSV files loading


In [ ]:
recipe_details = pd.read_csv("./culinaryDB/01_Recipe_Details.csv")
ingredients = pd.read_csv("./culinaryDB/02_Ingredients.csv")
compound_ingredients = pd.read_csv("./culinaryDB/03_Compound_Ingredients.csv")
recipe_ingredients= pd.read_csv("./culinaryDB/04_Recipe-Ingredients_Aliases.csv")

print(recipe_details.columns.tolist())

Costruzione dizionario id_ricetta : attributi

In [ ]:

node_attributes = {
    row["Recipe ID"]: {"Title": row["Title"], "Cuisine": row["Cuisine"]}
    for _, row in recipe_details.iterrows()
}

In [ ]:
node_attributes

Creazione del dizionario Ricetta : lista di ingredienti

In [ ]:
recipe_to_ingredients_dict = (
    recipe_ingredients
    .groupby("Recipe ID")["Aliased Ingredient Name"]
    .apply(list)
    .to_dict()
)







Costruzione mappa: ingrediente -> insieme di ricette che lo usano

In [ ]:
ingredient_to_recipes_dict = defaultdict(set)
for recipe_id, ingredients in recipe_to_ingredients_dict.items():
    for ing in ingredients:
        ingredient_to_recipes_dict[ing].add(recipe_id)

#ingredient_to_recipes_dict = dict(islice(ingredient_to_recipes_dict.items(), 10))

ingredient_to_recipes_dict

Inizializzazione grafo non orientato

In [ ]:
G = nx.Graph()

Crea archi: per ogni ingrediente, collega ricette che lo condividono

In [ ]:
# Inizializza lista per archi
edge_list = []
weight_list = []
edge_id = 0

count = 0
# Costruzione del grafo e raccolta dati per CSV
for ing, recipes in ingredient_to_recipes_dict.items():
    print(count + ing) # deve raggiungere
    for r1, r2 in combinations(recipes, 2):
        if G.has_edge(r1, r2):
            G[r1][r2]['ingredients'].add(ing)
            G[r1][r2]['weight'] += 1
        else:
            G.add_edge(r1, r2,ingredients={ing},  weight=1)
            edge_list.append([edge_id, r1, r2])
            weight_list.append([edge_id, 1])


Inserisco gli attributi ai nodi

In [ ]:
nx.set_node_attributes(G, node_attributes)

# verifica
print(G.nodes[1])
